# Data cleaning
We want the class with labels as numerical value and the body with clean text.

This will remove:
* duplicates
* NaN entires
* non english
* url, html

* make it lowercase
* combine title and body

In [1]:
import pandas as pd
import sys
sys.path.append("../../../scripts_shared/")
from preprocess_text import preprocess_text


In [2]:
# Read CSV into a dataframe
filename = "csv/rust_testset.csv"
df = pd.read_csv(filename)
df

,id,type,created_at,repo,repo_url,action,title,labels,body
0,1.141793e+10,IssuesEvent,2020-02-03 02:13:37,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,ICE due to `#[track_caller]` shim with -Z unle...,A-codegen A-const-eval A-const-fn F-track_call...,Reproducer:\r\n\r\n```rust\r\n// compile-flags...
1,1.141819e+10,IssuesEvent,2020-02-03 03:28:10,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,assertion failed: bpos.to_u32() >= mbc.pos.to_...,A-parser C-bug I-ICE I-nominated T-compiler,I'm seeing an internal compiler error on the f...
2,1.141898e+10,IssuesEvent,2020-02-03 06:38:37,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,ice on glacier 26459.rs with -Zsave-analysis,A-save-analysis C-bug I-ICE T-compiler,````rust\r\nfn main() {\r\n match 'a' {\r\n...
3,1.142033e+10,IssuesEvent,2020-02-03 09:54:09,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,Compiler performance returning nested closures,A-borrow-checker A-closures I-compiletime T-co...,The following takes approximately one year to ...
4,1.142279e+10,IssuesEvent,2020-02-03 14:49:22,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,ICE due to `#[track_caller]` shim with -Z unle...,A-codegen A-const-eval A-const-fn C-bug F-trac...,Reproducer:\r\n\r\n```rust\r\n// compile-flags...
...,...,...,...,...,...,...,...,...,...
25187,2.060418e+10,IssuesEvent,2022-03-06 18:28:40,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,ICE with generics.i,I-ICE T-compiler C-bug,\r\n\r\n### Code\r\n\r\nhttps://play.rust-lang...
25188,2.060495e+10,IssuesEvent,2022-03-06 20:51:18,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,discrepancy between debug and release: panic a...,C-bug,<!--\r\nThank you for filing a bug report! 🐛 P...
25189,2.060524e+10,IssuesEvent,2022-03-06 21:46:17,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,ICE: rustdoc GATs unwrap `None` #94009,I-ICE T-compiler C-bug,<!--\r\nThank you for finding an Internal Comp...
25190,2.060527e+10,IssuesEvent,2022-03-06 21:52:30,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,Compiler crash during doc test with generic_co...,I-ICE T-compiler C-bug,Trying to use `generic_const_exprs` (#76560) t...


In [3]:
# Number or different labels
df.labels.value_counts().to_frame()[:50]

,count
labels,
C-bug,3476
I-ICE T-compiler C-bug,1560
A-diagnostics T-compiler,1156
C-bug I-ICE T-compiler,784
A-docs,480
I-ICE,412
A-diagnostics,328
A-rustdoc,195
C-bug regression-untriaged,174


In [4]:
# Split data based on labels. Contains P0 and P1 in one dataframe, the rest in another

pattern = 'P-(high|critical)'
# Check if 'labels' contains the pattern
hp = df[df['labels'].str.contains(pattern)]
# Reset index
hp = hp.reset_index(drop=True)
hp


/tmp/ipykernel_12097/1172060366.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  hp = df[df['labels'].str.contains(pattern)]


,id,type,created_at,repo,repo_url,action,title,labels,body
0,1.142354e+10,IssuesEvent,2020-02-03 16:04:59,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,"ICE: 'rustc' panicked at 'Box<Any>', src/libru...",A-incr-comp C-bug E-needs-mcve I-ICE P-high T-...,"This may have a duplicate, since there are sim..."
1,7.237073e+09,IssuesEvent,2018-02-13 09:46:10,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,Incremental Compilation needs to rebuild every...,A-incr-comp C-bug P-high T-compiler,Given this example:\r\n\r\nmain.rs\r\n```rust\...
2,1.114081e+10,IssuesEvent,2019-12-21 17:17:43,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,matches macro conflicts with matches crate,A-macros A-resolve I-nominated P-high T-libs r...,cc https://github.com/rust-lang/rust/issues/65...
3,1.114085e+10,IssuesEvent,2019-12-21 17:35:10,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,ICE when using slice patterns in const fn.,A-const-eval A-slice-patterns C-bug F-slice_pa...,The following code ICEs\r\n\r\n```rust\r\n#![f...
4,5.088671e+09,IssuesEvent,2017-01-01 00:27:23,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,vendored dependendencies are incomplete in the...,I-nominated P-high regression-from-stable-to-b...,The process of creating the tarballs doesn't i...
...,...,...,...,...,...,...,...,...,...
855,4.099828e+09,IssuesEvent,2016-06-03 14:08:15,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,Glibc lock elision allows a value to be locked...,A-linux I-unsound P-high T-libs,When running on a processor which support Inte...
856,5.116484e+09,IssuesEvent,2017-01-07 04:02:29,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,ICE; two sources of truth on how stuff is retu...,I-ICE P-high regression-from-stable-to-stable ...,Consider this code:\r\n\r\n```rust\r\n#![featu...
857,5.116617e+09,IssuesEvent,2017-01-07 05:59:21,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,rustbuild builds docs for too many crates,A-rustbuild P-high regression-from-stable-to-b...,The Makefiles only build the docs for:\r\n\r\n...
858,7.802006e+09,IssuesEvent,2018-06-10 06:40:12,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,trivial_bounds doesn't see associated type as ...,P-high T-compiler regression-from-stable-to-ni...,Regression in rustc 1.27.0-nightly (2f2a11dfc ...


In [5]:
# Remove pattern from df
random = df[~df['labels'].str.contains(pattern)]
random = random.reset_index(drop=True)
random

/tmp/ipykernel_12097/2462673340.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  random = df[~df['labels'].str.contains(pattern)]


,id,type,created_at,repo,repo_url,action,title,labels,body
0,1.141793e+10,IssuesEvent,2020-02-03 02:13:37,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,ICE due to `#[track_caller]` shim with -Z unle...,A-codegen A-const-eval A-const-fn F-track_call...,Reproducer:\r\n\r\n```rust\r\n// compile-flags...
1,1.141819e+10,IssuesEvent,2020-02-03 03:28:10,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,assertion failed: bpos.to_u32() >= mbc.pos.to_...,A-parser C-bug I-ICE I-nominated T-compiler,I'm seeing an internal compiler error on the f...
2,1.141898e+10,IssuesEvent,2020-02-03 06:38:37,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,ice on glacier 26459.rs with -Zsave-analysis,A-save-analysis C-bug I-ICE T-compiler,````rust\r\nfn main() {\r\n match 'a' {\r\n...
3,1.142033e+10,IssuesEvent,2020-02-03 09:54:09,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,Compiler performance returning nested closures,A-borrow-checker A-closures I-compiletime T-co...,The following takes approximately one year to ...
4,1.142279e+10,IssuesEvent,2020-02-03 14:49:22,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,ICE due to `#[track_caller]` shim with -Z unle...,A-codegen A-const-eval A-const-fn C-bug F-trac...,Reproducer:\r\n\r\n```rust\r\n// compile-flags...
...,...,...,...,...,...,...,...,...,...
24327,2.060418e+10,IssuesEvent,2022-03-06 18:28:40,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,ICE with generics.i,I-ICE T-compiler C-bug,\r\n\r\n### Code\r\n\r\nhttps://play.rust-lang...
24328,2.060495e+10,IssuesEvent,2022-03-06 20:51:18,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,discrepancy between debug and release: panic a...,C-bug,<!--\r\nThank you for filing a bug report! 🐛 P...
24329,2.060524e+10,IssuesEvent,2022-03-06 21:46:17,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,ICE: rustdoc GATs unwrap `None` #94009,I-ICE T-compiler C-bug,<!--\r\nThank you for finding an Internal Comp...
24330,2.060527e+10,IssuesEvent,2022-03-06 21:52:30,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,Compiler crash during doc test with generic_co...,I-ICE T-compiler C-bug,Trying to use `generic_const_exprs` (#76560) t...


In [6]:
random.labels.value_counts().to_frame()[:50]

,count
labels,
C-bug,3476
I-ICE T-compiler C-bug,1560
A-diagnostics T-compiler,1156
C-bug I-ICE T-compiler,784
A-docs,480
I-ICE,412
A-diagnostics,328
A-rustdoc,195
C-bug regression-untriaged,174


In [7]:
# Number or different labels
hp.labels.value_counts().to_frame()[:50]

,count
labels,
P-high T-compiler,14
P-high T-compiler regression-from-stable-to-beta,13
P-high,10
P-high regression-from-stable-to-beta T-compiler,9
I-ICE P-high regression-from-stable-to-nightly T-compiler,6
C-bug I-ICE P-high T-compiler regression-from-stable-to-stable,6
P-high T-compiler regression-from-stable-to-stable,6
C-bug P-high T-compiler regression-from-stable-to-beta,6
regression-from-stable-to-nightly P-high T-compiler C-bug,6


In [8]:
#Give each priority a label by number.
# 'Label encoding'. Makes is easier for machine learning models to work with categorical data.
hp["label"] = 1
hp["class"] = "high_priority"
hp.head()

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
0,1.142354e+10,IssuesEvent,2020-02-03 16:04:59,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,"ICE: 'rustc' panicked at 'Box<Any>', src/libru...",A-incr-comp C-bug E-needs-mcve I-ICE P-high T-...,"This may have a duplicate, since there are sim...",1,high_priority
1,7.237073e+09,IssuesEvent,2018-02-13 09:46:10,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,Incremental Compilation needs to rebuild every...,A-incr-comp C-bug P-high T-compiler,Given this example:\r\n\r\nmain.rs\r\n```rust\...,1,high_priority
2,1.114081e+10,IssuesEvent,2019-12-21 17:17:43,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,matches macro conflicts with matches crate,A-macros A-resolve I-nominated P-high T-libs r...,cc https://github.com/rust-lang/rust/issues/65...,1,high_priority
3,1.114085e+10,IssuesEvent,2019-12-21 17:35:10,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,ICE when using slice patterns in const fn.,A-const-eval A-slice-patterns C-bug F-slice_pa...,The following code ICEs\r\n\r\n```rust\r\n#![f...,1,high_priority
4,5.088671e+09,IssuesEvent,2017-01-01 00:27:23,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,vendored dependendencies are incomplete in the...,I-nominated P-high regression-from-stable-to-b...,The process of creating the tarballs doesn't i...,1,high_priority


In [9]:
random["label"] = 0
random["class"] = "not_high_priority" 
random.head()

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
0,1.141793e+10,IssuesEvent,2020-02-03 02:13:37,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,ICE due to `#[track_caller]` shim with -Z unle...,A-codegen A-const-eval A-const-fn F-track_call...,Reproducer:\r\n\r\n```rust\r\n// compile-flags...,0,not_high_priority
1,1.141819e+10,IssuesEvent,2020-02-03 03:28:10,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,assertion failed: bpos.to_u32() >= mbc.pos.to_...,A-parser C-bug I-ICE I-nominated T-compiler,I'm seeing an internal compiler error on the f...,0,not_high_priority
2,1.141898e+10,IssuesEvent,2020-02-03 06:38:37,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,ice on glacier 26459.rs with -Zsave-analysis,A-save-analysis C-bug I-ICE T-compiler,````rust\r\nfn main() {\r\n match 'a' {\r\n...,0,not_high_priority
3,1.142033e+10,IssuesEvent,2020-02-03 09:54:09,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,Compiler performance returning nested closures,A-borrow-checker A-closures I-compiletime T-co...,The following takes approximately one year to ...,0,not_high_priority
4,1.142279e+10,IssuesEvent,2020-02-03 14:49:22,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,ICE due to `#[track_caller]` shim with -Z unle...,A-codegen A-const-eval A-const-fn C-bug F-trac...,Reproducer:\r\n\r\n```rust\r\n// compile-flags...,0,not_high_priority


In [10]:
# Drop duplicates by the content of the title
high_priority = hp.drop_duplicates(subset=['title'], keep='last')
high_priority.dropna(inplace=True)
high_priority.reset_index(inplace=True)
high_priority.drop(columns=["index"] , inplace= True)
high_priority["class"].value_counts()

/tmp/ipykernel_12097/1823776403.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_priority.dropna(inplace=True)
/tmp/ipykernel_12097/1823776403.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_priority.drop(columns=["index"] , inplace= True)


class
high_priority    788
Name: count, dtype: int64

In [11]:
# Drop duplicates by the content of the title
not_high_priority = random.drop_duplicates(subset=['title'], keep='last')
not_high_priority.dropna(inplace=True)
not_high_priority.reset_index(inplace=True)
not_high_priority.drop(columns=["index"] , inplace= True)
not_high_priority["class"].value_counts()

/tmp/ipykernel_12097/3923262596.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_high_priority.dropna(inplace=True)
/tmp/ipykernel_12097/3923262596.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_high_priority.drop(columns=["index"] , inplace= True)


class
not_high_priority    19865
Name: count, dtype: int64

In [12]:
high_priority.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788 entries, 0 to 787
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          788 non-null    float64
 1   type        788 non-null    object 
 2   created_at  788 non-null    object 
 3   repo        788 non-null    object 
 4   repo_url    788 non-null    object 
 5   action      788 non-null    object 
 6   title       788 non-null    object 
 7   labels      788 non-null    object 
 8   body        788 non-null    object 
 9   label       788 non-null    int64  
 10  class       788 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 67.8+ KB


In [13]:
label_counts = high_priority["class"].value_counts()
label_counts_nhp = not_high_priority["class"].value_counts()
print(label_counts)
not_high_priority_count = label_counts_nhp["not_high_priority"]
print(not_high_priority_count)
hp_count = label_counts["high_priority"]
hp_count

class
high_priority    788
Name: count, dtype: int64
19865


788

In [14]:
not_high_priority = not_high_priority.sample(frac=hp_count/not_high_priority_count, random_state=42)
not_high_priority

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
8387,2.025647e+10,IssuesEvent,2022-02-15 00:03:59,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,bootstrap: RUSTFLAGS_*_BOOTSTRAP does not affe...,C-bug,It's unclear if this _should_ be a bug report ...,0,not_high_priority
4564,2.625521e+10,IssuesEvent,2023-01-05 23:39:34,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,"Unexpected ""expected function"" error with unit...",C-bug,<!--\r\nThank you for filing a bug report! 🐛 P...,0,not_high_priority
19348,1.357796e+10,IssuesEvent,2020-09-20 04:46:50,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,The default overflow behavior of u128 arithmet...,C-bug,<!--\r\nThank you for filing a bug report! 🐛 P...,0,not_high_priority
13702,6.997215e+09,IssuesEvent,2017-12-16 11:54:18,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,Assertion failed in rustc : !(self.has_self &&...,A-resolve C-bug I-ICE,When using `cargo build` to build one of my pr...,0,not_high_priority
9419,2.491293e+10,IssuesEvent,2022-10-30 03:32:39,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,ICE: assertion failed: elem.index() < self.dom...,I-ICE T-compiler C-bug,### Code\r\n\r\n```Rust\r\nstruct Apple((Apple...,0,not_high_priority
...,...,...,...,...,...,...,...,...,...,...,...
15458,3.070998e+10,IssuesEvent,2023-07-27 09:06:05,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,Suggest `by_ref` when taking a reference to an...,A-diagnostics T-compiler,### Code\n\n```Rust\nfn main() {\r\n let mu...,0,not_high_priority
17656,7.327034e+09,IssuesEvent,2018-03-04 04:15:21,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,Type inference regression affecting app_dirs o...,I-nominated T-libs regression-from-stable-to-beta,As seen in [this Travis build](https://travis-...,0,not_high_priority
6153,1.247528e+10,IssuesEvent,2020-05-29 11:16:43,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,Impl Error for Infallible,A-error-handling C-enhancement T-libs,Is there a reason why `std::convert::Infallibl...,0,not_high_priority
3633,7.748851e+09,IssuesEvent,2018-05-30 09:33:59,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,resolve suggestions do not use `crate::` when ...,A-diagnostics A-rust-2018-preview T-compiler W...,When you get name resolution suggestions from ...,0,not_high_priority


In [15]:
all_priority = pd.concat([high_priority,not_high_priority] , ignore_index = True)
all_priority.tail()

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
1571,3.070998e+10,IssuesEvent,2023-07-27 09:06:05,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,Suggest `by_ref` when taking a reference to an...,A-diagnostics T-compiler,### Code\n\n```Rust\nfn main() {\r\n let mu...,0,not_high_priority
1572,7.327034e+09,IssuesEvent,2018-03-04 04:15:21,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,Type inference regression affecting app_dirs o...,I-nominated T-libs regression-from-stable-to-beta,As seen in [this Travis build](https://travis-...,0,not_high_priority
1573,1.247528e+10,IssuesEvent,2020-05-29 11:16:43,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,Impl Error for Infallible,A-error-handling C-enhancement T-libs,Is there a reason why `std::convert::Infallibl...,0,not_high_priority
1574,7.748851e+09,IssuesEvent,2018-05-30 09:33:59,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,resolve suggestions do not use `crate::` when ...,A-diagnostics A-rust-2018-preview T-compiler W...,When you get name resolution suggestions from ...,0,not_high_priority
1575,1.222668e+10,IssuesEvent,2020-05-03 12:05:44,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,ICE: rmeta/decoder.rs:1130: impossible case re...,A-metadata C-bug E-needs-mcve I-ICE T-compiler,<!--\r\nThank you for finding an Internal Comp...,0,not_high_priority


In [16]:
# 
all_priority["label"].value_counts()

label
1    788
0    788
Name: count, dtype: int64

In [17]:
print(all_priority["title"][0])
print(all_priority["body"][0])

ICE: 'rustc' panicked at 'Box<Any>', src/librustc_errors/lib.rs:931:9
This may have a duplicate, since there are similar panics already reported, but this is on a different line number.

The compilation log and source code are listed below

```rust
pub mod client;
pub mod msg;

use std::collections::HashMap;
use std::sync::Arc;
use tokio::sync::broadcast::{channel, Receiver, Sender};

/**
 * A channel is named and typed with the type of messages it should be carrying
 */
struct Channel {
    name: String,
    send: Sender<Message>,
    recv: Receiver<Message>,
}

struct Message {}

impl Channel {
    fn send(&self, msg: Message) {}
    fn recv(&self, msg: Message) {}
}

struct Bus {
    /**
     * Channels are named and can implement a number of different types. This should
     * allow the Bus to handle different channels with different message payloads
     * while still taking advantage of compile-time checks
     */
    channels: HashMap<String, Channel>,
}

#[cfg(test)]
mod tests 

In [18]:
# Copy content of body to a new col named text
all_priority["text"] = all_priority["title"] + " " + all_priority["body"]
all_priority.tail()

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class,text
1571,3.070998e+10,IssuesEvent,2023-07-27 09:06:05,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,Suggest `by_ref` when taking a reference to an...,A-diagnostics T-compiler,### Code\n\n```Rust\nfn main() {\r\n let mu...,0,not_high_priority,Suggest `by_ref` when taking a reference to an...
1572,7.327034e+09,IssuesEvent,2018-03-04 04:15:21,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,Type inference regression affecting app_dirs o...,I-nominated T-libs regression-from-stable-to-beta,As seen in [this Travis build](https://travis-...,0,not_high_priority,Type inference regression affecting app_dirs o...
1573,1.247528e+10,IssuesEvent,2020-05-29 11:16:43,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,Impl Error for Infallible,A-error-handling C-enhancement T-libs,Is there a reason why `std::convert::Infallibl...,0,not_high_priority,Impl Error for Infallible Is there a reason wh...
1574,7.748851e+09,IssuesEvent,2018-05-30 09:33:59,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,opened,resolve suggestions do not use `crate::` when ...,A-diagnostics A-rust-2018-preview T-compiler W...,When you get name resolution suggestions from ...,0,not_high_priority,resolve suggestions do not use `crate::` when ...
1575,1.222668e+10,IssuesEvent,2020-05-03 12:05:44,rust-lang/rust,https://api.github.com/repos/rust-lang/rust,closed,ICE: rmeta/decoder.rs:1130: impossible case re...,A-metadata C-bug E-needs-mcve I-ICE T-compiler,<!--\r\nThank you for finding an Internal Comp...,0,not_high_priority,ICE: rmeta/decoder.rs:1130: impossible case re...


In [19]:
all_priority["text"][0]

"ICE: 'rustc' panicked at 'Box<Any>', src/librustc_errors/lib.rs:931:9 This may have a duplicate, since there are similar panics already reported, but this is on a different line number.\r\n\r\nThe compilation log and source code are listed below\r\n\r\n```rust\r\npub mod client;\r\npub mod msg;\r\n\r\nuse std::collections::HashMap;\r\nuse std::sync::Arc;\r\nuse tokio::sync::broadcast::{channel, Receiver, Sender};\r\n\r\n/**\r\n * A channel is named and typed with the type of messages it should be carrying\r\n */\r\nstruct Channel {\r\n    name: String,\r\n    send: Sender<Message>,\r\n    recv: Receiver<Message>,\r\n}\r\n\r\nstruct Message {}\r\n\r\nimpl Channel {\r\n    fn send(&self, msg: Message) {}\r\n    fn recv(&self, msg: Message) {}\r\n}\r\n\r\nstruct Bus {\r\n    /**\r\n     * Channels are named and can implement a number of different types. This should\r\n     * allow the Bus to handle different channels with different message payloads\r\n     * while still taking advantage 

In [20]:
# Make a new dataframe with only text, label and class cols.
all_priority_subset = all_priority[["text" , "label" , "class"]]
all_priority_subset

,text,label,class
0,"ICE: 'rustc' panicked at 'Box<Any>', src/libru...",1,high_priority
1,Incremental Compilation needs to rebuild every...,1,high_priority
2,matches macro conflicts with matches crate cc ...,1,high_priority
3,ICE when using slice patterns in const fn. The...,1,high_priority
4,vendored dependendencies are incomplete in the...,1,high_priority
...,...,...,...
1571,Suggest `by_ref` when taking a reference to an...,0,not_high_priority
1572,Type inference regression affecting app_dirs o...,0,not_high_priority
1573,Impl Error for Infallible Is there a reason wh...,0,not_high_priority
1574,resolve suggestions do not use `crate::` when ...,0,not_high_priority


In [21]:
# Convert to string
all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)

/tmp/ipykernel_12097/3106036705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)


In [22]:
all_priority_subset

,text,label,class,text_str
0,"ICE: 'rustc' panicked at 'Box<Any>', src/libru...",1,high_priority,"ICE: 'rustc' panicked at 'Box<Any>', src/libru..."
1,Incremental Compilation needs to rebuild every...,1,high_priority,Incremental Compilation needs to rebuild every...
2,matches macro conflicts with matches crate cc ...,1,high_priority,matches macro conflicts with matches crate cc ...
3,ICE when using slice patterns in const fn. The...,1,high_priority,ICE when using slice patterns in const fn. The...
4,vendored dependendencies are incomplete in the...,1,high_priority,vendored dependendencies are incomplete in the...
...,...,...,...,...
1571,Suggest `by_ref` when taking a reference to an...,0,not_high_priority,Suggest `by_ref` when taking a reference to an...
1572,Type inference regression affecting app_dirs o...,0,not_high_priority,Type inference regression affecting app_dirs o...
1573,Impl Error for Infallible Is there a reason wh...,0,not_high_priority,Impl Error for Infallible Is there a reason wh...
1574,resolve suggestions do not use `crate::` when ...,0,not_high_priority,resolve suggestions do not use `crate::` when ...


In [23]:
# Clean the data.
all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)

/tmp/ipykernel_12097/63294665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)


In [24]:
# Make a subset with text_clean and label
priority_label_text = all_priority_subset[["text_clean" , "label"]]
priority_label_text

,text_clean,label
0,ice rustc panicked at box this may have a dupl...,1
1,incremental compilation needs to rebuild every...,1
2,matches macro conflicts with matches crate cc ...,1
3,ice when using slice patterns in const fn the ...,1
4,vendored dependendencies are incomplete in the...,1
...,...,...
1571,suggest byref when taking a reference to an it...,0
1572,type inference regression affecting appdirs on...,0
1573,impl error for infallible is there a reason wh...,0
1574,resolve suggestions do not use crate when enab...,0


In [25]:
# Need to dropna here since cleaning function returns NaN for not english text.
priority_label_text.dropna(inplace=True)
priority_label_text.reset_index(inplace=True)
priority_label_text.drop(columns=["index"] , inplace= True)

priority_label_text

/tmp/ipykernel_12097/239147930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.dropna(inplace=True)
/tmp/ipykernel_12097/239147930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.drop(columns=["index"] , inplace= True)


,text_clean,label
0,ice rustc panicked at box this may have a dupl...,1
1,incremental compilation needs to rebuild every...,1
2,matches macro conflicts with matches crate cc ...,1
3,ice when using slice patterns in const fn the ...,1
4,vendored dependendencies are incomplete in the...,1
...,...,...
1568,suggest byref when taking a reference to an it...,0
1569,type inference regression affecting appdirs on...,0
1570,impl error for infallible is there a reason wh...,0
1571,resolve suggestions do not use crate when enab...,0


In [26]:
from sklearn.utils import resample

# Split the dataset into two based on the label
df_majority = priority_label_text[priority_label_text['label'] == 0]
df_minority = priority_label_text[priority_label_text['label'] == 1]

# Undersample the majority class
df_majority_undersampled = resample(df_majority, 
                                     replace=False,    # sample without replacement
                                     n_samples=len(df_minority),     # to match minority class
                                     random_state=123) # reproducible results

# Combine minority class with downsampled majority class
df_balanced = pd.concat([df_majority_undersampled, df_minority])

# Shuffle the dataset to avoid any ordering bias
df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)
priority_label_text = df_balanced
priority_label_text["label"].value_counts()

label
0    786
1    786
Name: count, dtype: int64

In [27]:
# Clean dataset with clean text and labels.
# 0 = high priority, 1 = not high priority
file_name = f"csv/clean_rust_testset.csv"
priority_label_text.to_csv(file_name, index=False)

In [28]:
pri = pd.read_csv(file_name)
pri

,text_clean,label
0,verify that backported commits are backports t...,0
1,two different versions of a crate interacting ...,0
2,ice when using impl trait with an infinity rec...,1
3,compiler error while compiling winrt trying to...,1
4,rust segmentation fault related to const gener...,1
...,...,...
1567,tracking issue for stdfs extensions rfc,1
1568,assertion failed error on trying to bench with...,1
1569,ice caused by llvm assertion assertion sgettyp...,0
1570,add a build container for the target was recen...,0
